### Initialization

In [52]:
import sys
import importlib
import importlib.util

import tika
from tika import parser
from tika import language
from tika import config

import pdf2image

import pytesseract
import tabula
from pathlib import Path
import os

### Text extraction

In [138]:
class ExtractionMachine:
    
    _extractor = None
    _extractions = {}
    _save = None
    _filter_extracted = None
    
    def __init__(self, extractor=CombinedDocExtractor(), save=False, filter_extracted=False):
        self._extractor = extractor
        self._save = save
        self._filter_extracted = filter_extracted
    
    def extractFromDirs(self, dirs):
        for path in dirs:
            self.extractFromDir(path)
        return self._extractions
    
    def extractFromDir(self, path):
        filenames = []
        if os.path.isfile(path):
            filenames.append(path)
        else:
            filenames = [p.as_posix() for p in Path(path).rglob('*.*')]
        if self._filter_extracted:
            filenames = [p for p in filenames if not p.endswith('_ext')]
        for i, f in enumerate(filenames):
            print('{}/{}: {}'.format(i+1, len(filenames), f))
            self.extractDocument(f)
        return self._extractions
    
    def extractDocument(self, path):
        text = self._extractor.extract(path)
        if self._save:
            self.saveExtract(path, text)
        else:
            self._extractions[path] = text
            
    def saveExtract(self, path, text):
        with open(path+'_ext', 'w', encoding='utf-8') as f:
            f.write(text)

In [143]:
class DocumentExtractor:

    _core = None
    
    def __init__(self, extractor):
        self._core = extractor
        
    def extract(self, path):
        data = self._preprocess(path)
        return self._process(data)
        
    def _preprocess(self, path):
        pass
    
    def _process(self, data):
        pass
    
class TikaDocExtractor(DocumentExtractor):
    
    def __init__(self):
        super().__init__(TikaTextExtractor())
        
    def _preprocess(self, path):
        return path
    
    def _process(self, path):
        return self._core.extract(path)

class PytesseractDocExtractor(DocumentExtractor):
    
    _dpi = None
    
    def __init__(self, dpi=300):
        super().__init__(PytesseractTextExtractor())
        if dpi is not None:
            self._dpi = dpi
        
    def _preprocess(self, path):
        return pdf2image.convert_from_path(path, self._dpi)
    
    def _process(self, pages):
        extractions = []
        for page in pages:
            extractions.append(self._core.extract(page))
        text = ''.join(extractions)            
        return text
    
class CombinedDocExtractor():
    
    _tika_core = None
    _pytesseract_core = None
    
    def __init__(self):
        self._tika_core = TikaDocExtractor()
        self._pytesseract_core = PytesseractDocExtractor()
    
    def extract(self, path):
        text = self._tika_core.extract(path)
        if text is None and path.lower().endswith('.pdf'):
            text = self._pytesseract_core.extract(path)
        return text
    
class CombinedDocExtractor2(CombinedDocExtractor):
    
    def extract(self, path):
        if path.lower().endswith('.pdf'):
            text = self._pytesseract_core.extract(path)
        else:
            text = self._tika_core.extract(path)
        return text

In [140]:
class TextExtractor:
    
    def __init__(self):
        pass
        
class TikaTextExtractor(TextExtractor):
    
    def extract(self, path):
        data = tika.parser.from_file(path, headers={'X-Tika-OCRLanguage': 'ces'})
        if data['status'] == 200:
            return data['content']
        else:
            return ''
    
class PytesseractTextExtractor(TextExtractor):
    
    def extract(self, image):
        return pytesseract.image_to_string(image, lang='ces')

### Playgroung

In [5]:
tikaExtractor = TikaTextExtractor()

In [6]:
tesseractExtractor = PytesseractExtractionMachine()

In [7]:
machine = CombinedExtractionMachine()

In [14]:
df = tabula.read_pdf('../dokumenty/test_src/test_sub_01/test_contr_02/smlouva.pdf')

In [10]:
filename = '../dokumenty/test_src/test_sub_01/test_contr_02/smlouva.pdf'
file_data = parser.from_file(filename, headers={'X-Tika-OCRLanguage': 'ces'})

2019-11-14 17:00:24,511 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


In [3]:
filename = '../dokumenty/test_src/test_sub_01/test_contr_03/zeman.pdf'

In [4]:
parser.from_file(filename)

{'status': 200,
 'content': '\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMiloš Zeman (* 28. září 1944 Kolín) je český politik, ekonom, prognostik a třetí prezident \n\nČeské republiky. Úřadu se poprvé ujal složením slibu 8. března 2013 a druhé funkční období \n\nzahájil přesně o pět let později.  \n\nV období 1993–2001 působil jako předseda České strany sociálně demokratické. V letech \n\n1996–1998 zastával funkci předsedy Poslanecké sněmovny Parlamentu České republiky \n\na následující čtyři roky byl předsedou sociálnědemokratické vlády menšinového charakteru, \n\njejíž existence byla umožněna na základě tzv. „opoziční smlouvy“ s Občanskou \n\ndemokratickou stranou.  \n\nPo neúspěšné kandidatuře na prezidenta České republiky v roce 2003 odešel na sedm let \n\nz politiky. V březnu 2010 se stal předsedou nově založené Strany Práv Občanů \n\nZEMANOVCI. Na funkci rezignoval po parlamentních volbách 2010, ve kterých se strana \n\nnedostala do sně

In [45]:
filename = '../dokumenty/test_src/test_sub_01/test_contr_04/podepsaná smlouva projekt střecha RK.pdf'

In [53]:
machine.extract(filename)

'„e 16752/2016-MZE-12134\n\n- JÍ | M\nMINISTERSTVO ZMĚDĚLSTY DU\n44093426\n\n0002\n\nSmlouva o dílo č. 256-2016-12134\nuzavřená podle $ 2586 a násl. zákona č. 89/2012 Sb., občanského zákoníku (dále jen\n„občanský zákoník“) ve spojení s 8 2631 občanského zákoníku\n\nmezi\n\nČeská republika — Ministerstvo zemědělství\nse sídlem: Těšnov 65/17, 110 00 Praha 1 — Nové Město\n\nIČ: 00020478\n\nDIČ: CZ00024478\n\nBankovní spojení: Česká národní banka Praha 1\n\nČíslo účtu: 1226-984418740  7226007/0£70 | OPRRVAVO LEG. BE /\nzastoupenou Ing. Jiří Boháčkem, ředitelem odboru vnitřní správy\n\n(dále jen „objednatel“)\n\na společností\n\nD-PLUS PROJEKTOVÁ A INŽENÝRSKÁ a.s.\n\nse sídlem: Sokolovská 45/16, 186 00 Praha 8\n\nIČ: 26760312\n\nDIČ: CZ26760312\n\nSpolečnost je registrována u Městského soudu v Praze, oddíl B, vložka 8111\nBankovní spojení: ČSOB a.s., Praha 1\n\nčíslo účtu: 581094323/0300\n\nzastoupenou jednatelem společnosti\n\n(dále jen „zhotovitel “ nebo „projektant“)\n\nČlánek 1\nPředmět

In [14]:
filename = '../dokumenty/test_src/test_sub_01/test_contr_04/podepsanĂˇ smlouva projekt stĹ™echa RK-01.jpg'

In [15]:
res = tikaExtractor.extract(filename)

In [16]:
res

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n16752/2016-MZE-12134\n\np oso MAN\n000244093426\n\n  \n\n|\n\n|\n\n|\n\nSmlouva o dílo č. 256-2016-12134\nuzavřená podle $ 2586 a násl. zákona č. 89/2012 Sb., občanského zákoníku (dále jen\n\n„občanský zákoník“) ve spojení s $ 2631 občanského zákoníku\n\nmezi\n\nČeská republika — Ministerstvo zemědělství\nse sídlem: Těšnov 65/17, 110 00 Praha 1 — Nové Město\n\nIČ: 00020478\n\nDIČ: CZ00024478\n\nBankovní spojení: Česká národní banka Praha 1\n\nČíslo účtu: 1226-901118740 7226007/0270  OPRAVÁVO ŽE DVE\nzastoupenou Ing. Jiří Boháčkem, ředitelem odboru vnitřní správy\n\n(dále jen „objednatel“)\n\na společností\n\nD-PLUS PROJEKTOVÁ A INŽENÝRSKÁ a.s.\n\nse sídlem: Sokolovská 45/16, 186 00 Praha 8\n\nIČ: 26760312\n\nDIČ: CZ26760312\n\nSpolečnost je registrována u Městského soudu v Praze, oddíl B, vložka 8111\nBankovní spojení: ČSOB a.s., Praha 1\n\nčíslo účtu: 581094323/0300\n\nzastoupenou jednatelem společnosti\n\n(dále jen „zhotovite

In [24]:
filenames = glob.glob('../dokumenty/test_src/test_sub_01/test_contr_04/*.jpg')

In [36]:
%%time

for filename in filenames:
    display(parser.from_file(filename, headers={'X-Tika-OCRLanguage': 'ces'})['content'])

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n16752/2016-MZE-12134\n\np oso MAN\n000244093426\n\n  \n\n|\n\n|\n\n|\n\nSmlouva o dílo č. 256-2016-12134\nuzavřená podle $ 2586 a násl. zákona č. 89/2012 Sb., občanského zákoníku (dále jen\n\n„občanský zákoník“) ve spojení s $ 2631 občanského zákoníku\n\nmezi\n\nČeská republika — Ministerstvo zemědělství\nse sídlem: Těšnov 65/17, 110 00 Praha 1 — Nové Město\n\nIČ: 00020478\n\nDIČ: CZ00024478\n\nBankovní spojení: Česká národní banka Praha 1\n\nČíslo účtu: 1226-901118740 7226007/0270  OPRAVÁVO ŽE DVE\nzastoupenou Ing. Jiří Boháčkem, ředitelem odboru vnitřní správy\n\n(dále jen „objednatel“)\n\na společností\n\nD-PLUS PROJEKTOVÁ A INŽENÝRSKÁ a.s.\n\nse sídlem: Sokolovská 45/16, 186 00 Praha 8\n\nIČ: 26760312\n\nDIČ: CZ26760312\n\nSpolečnost je registrována u Městského soudu v Praze, oddíl B, vložka 8111\nBankovní spojení: ČSOB a.s., Praha 1\n\nčíslo účtu: 581094323/0300\n\nzastoupenou jednatelem společnosti\n\n(dále jen „zhotovite

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\njen „DPS“) dle S 1 odst. 1 písm. f) vyhlášky 499/2006 Sb. (bude použita pro výběr\ndodavatele stavby) to vše na investiční akci „Projektová činnost pro rekonstrukci\nstřešního pláště budovy Jiráskova 1320 Rychnov nad Kněžnou“, autorský dozor výše\nuvedené investiční akce a zajištění průkazu energetické náročnosti budovy v souladu\ns příslušnými ustanoveními zákona č. 183/2006 Sb., o územním plánování a stavebním\nřádu (stavební zákon), ve znění pozdějších předpisů (dále jen „stavební zákon“), včetně\nprávních předpisů, které stavební zákon provádějí (veškeré výše uvedené společně dále\njen jako „dílo“).\n\nRozsah a etapy díla:\n\n1. Stavebně technické průzkumy, digitalizace a ověření stávajícího stavu, zpracování\nkoncepce řešení, DSP (V rozsahu pro výběr zhotovitele stavby — položkový rozpočet a\nvýkaz výměr) včetně pravomocného stavebního povolení, hodnocení ekonomického\npřínosu fotovoltaické folie\n\n1.1. Zajištění stavebně

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nzhotovitele stavby, kdy bude povinností projektanta posouzení rozpočtů v nabídkách\nuchazečů — zejména z pohledu problematiky mimořádně nízké nabídkové ceny, zda\njsou řádně vyplněny všechny položky atp.\n\n1.5. Autorský dozor projektanta v průběhu výstavby (dále jen „AD“)\n\nVýkon autorského dozoru projektanta bude spočívat zejména v poskytování\nvysvětlení potřebných pro vypracování dodavatelské dokumentace nebo pro plynulost\nstavby, dohledu nad dodržením dokumentace pro provádění stavby s přihlédnutím\nk podmínkám stavebního povolení a posuzování návrhů dodavatelů na změny a\nodchylky oproti této dokumentaci, zejména z pohledu dodržení technicko-\nekonomických parametrů stavby, architektonické koncepce, v povinné účasti na\nodevzdání a převzetí stavby, v povinné účasti na kontrolních dnech stavby apod.\n\nSoučástí ceny je tak nepřetržité provádění AD, a to do předání stavebního\ndíla/stavby, které je předmětem autorského do

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nVypracování projektantem 6 paré kompletní projektové dokumentace DSP a zajištění\nstavebního povolení (IČ) (ve smyslu nabytí právní moci) projektantem (dále jen jako\n„část díla 2“).\n\nVypracování projektantem 6 paré projektové dokumentace DPS (dále jen jako „část díla\n3).\n\nZajištění kolaudačního souhlasu projektantem (IČ) (ve smyslu nabytí právní moci tohoto\nrozhodnutí) a autorského dozoru projektanta (AD) v rozsahu a délce trvání uvedené\nv 1.5. (dále jen jako „část díla 4“).\n\nČlánek 2\nPovinnosti objednatele\n\n2.1. Objednatel se zavazuje, že poskytne zhotoviteli potřebnou součinnost při plnění této\nsmlouvy.\n\n2.2. Objednatel se zavazuje předat zhotoviteli všechny nezbytné podklady vztahující se\nk řešenému území.\n\n2.3. Případná vyjádření k projektům v průběhu prací se zavazuje objednatel předat\nzhotoviteli v přiměřených lhůtách, nikoliv však kratší Ihůtě než 5 pracovních dní\n\n2.4. Objednatel se zavazuje oznámi

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nv 1.5. Smlouvy.\n\n3.5. V případě, že v rámci kontroly předjímané v čl. 3.4. bude konstatováno objednatelem\nporušení některé z povinností projektanta, předjímané smluvně či zákonem, pořídí o této\nskutečnosti objednatel zápis, jehož součástí může být i lhůta stanovená objednatelem\npro odstranění výše uvedené porušované povinnosti.\n\nČlánek 4\nDoba plnění\n\n4.1. Zhotovitel se zavazuje, že dílo, resp. jednotlivé části díla budou převzaty\nobjednatelem v souladu s touto Smlouvou v termínech níže uvedených a definovaných.\n\na) část díla 1 - do dvou týdnů od podpisu smlouvy druhou smluvní stranou\n\nb) část díla 2 - do 2 měsíců od podpisu smlouvy s podáním stavebního povolení\nna příslušné úřady\n\nc) část díla 3 - do jednoho měsíce od vydání pravomocného rozhodnutí o vydání\nstavebního povolení\n\nd) část díla 4 — do vydání pravomocného kolaudačního souhlasu, autorský dozor\npo dobu výstavby\n\n4.2. Termíny mohou být upraveny 

...

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nv rozpracovaném stavu, a veškeré související dokumentace, objednatelem za podmínek\nstanovených touto smlouvou a autorským zákonem. Právo dílo užít podle předchozí věty\nzahrnuje všechna oprávnění dle ustanovení $ 12 zákona č. 121/2000 Sb., o právu\nautorském, o právech souvisejících s právem autorským a o změně některých zákonů\n(autorský zákon). Zhotovitel se zavazuje, že sám nepoužije ani neposkytne žádné třetí\nosobě bez předchozího písemného souhlasu objednatele práva kužití díla, resp.\njakékoliv části díla, provedeného dle této smlouvy. Právo dílo užít v rozsahu výše\nuvedeném vzniká objednateli předáním díla. Předáním dílčích částí díla vzniká\nobjednateli právo užít dílčí část díla v rozsahu předání této dílčí části.\n\n10.2. Smluvní strany sjednávají, že vlastnické právo k technickým dokumentacím, které\ntvoří součást díla, resp. část díla, jakož i všechny ostatní hmotné podklady, na kterých je\ndílo, resp. část díla 

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nV případě objednatele:\n\nČeská republika - Ministerstvo zemědělství\n\nTěšnov 65/17 , Praha 1 — Nové Město\n\nIng. Jiří Boháček, ředitel odboru vnitřní správy\n\ne-mail: jiri.bohacekAmze.cz\n\nV případě zhotovitele:\n\nD-PLUS PROJEKTOVÁ A INŽENÝRSKÁ a.s.\n\nSokolovská 45/16, Praha 8\n\nve věcech smluvních: Ing. Karel Janoch, předseda představenstva\n\ne-mail: karel.janoch©Ad-plus.cz\n\nve věcech technických: Ing. Karel Janoch, předseda představenstva\n\ne-mail: karel.janoch ©Wd-plus.cz\n\nnebo na takovou jinou adresu, která bude smluvní straně, která je odesílatelem\noznámení, předem oznámena v souladu s tímto článkem.\n\nPřílohy:\n\nč. 1— Prohlášení autora projektové dokumentace DSP, DPS\n\nza zhotovitele\n\nd plus\n\nAROJEKTOVÁ A mževýnaná A2 ©\n:OKOLOVSKÁ 16/45A.\n186 00 PRAHA 8\n\n \n\n16752/2016-MZE-12134\n\n-7 04, 2006\n\nV Praze, dne- K3\n\nza objednatele\nMINISTERSTVO\nZEMĚDĚLSTVÍ\n\n110.00 Prahz\n\n  \n\n   \n\nČeská 

"\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nProhlášení autora a výhradní bezúplatná licenční smlouva pro užití v neomezeném rozsahu\n\nJá, níže podepsaný: Ing. Karel Janoch,\n\nnar. 10. 7. 1964, bytem Černokostelecká 101/32, Praha 10, místem podnikání\nRČ 6407101910\n\n(dále jen „Autor“)\n\ntímto ve smyslu z.č. 89/2012 Sb., občanského zákoníku (dále jen „NOZ“), zejména ust. $ 2358 a násl.\na $ 2950 NOZ, a z.č. 121/2000 Sb., o právu autorském, o právech souvisejících s právem autorským\na o změně některých zákonů (dále jen „AZ“), zejména ust. $ 9 a násl. , $ 58 a/nebo $ 61 AZ,\n\nprohlašuji, že\n\n-jsem v rámci pracovněprávního vztahu k firmě D-PLUS PROJEKTOVÁ A INŽENÝRSKÁ a.s.\nvytvořil zaměstnanecké je Ji S 58 AZ a sohledem na smlouvu o dílo č.j 16752-2016-\nMZE12134 ze dne. uzavřenou mezi Českou republikou - Ministerstvem\nzemědělství, se sídlem m 17, Praha — Nové Město, PSČ: 110 00, IČ: 00020478 (dále jen\n„MZe“) a zhotovitelem vybraným v rámci VZMR „Projektová činnos

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n  \n \n\nprojekty a že spoléhá na pravost, úplnost, platnost a dlouhodobou účinnost tohoto prohlášení\n\nV Praze dne ... 20. 04,206\n\n \n\nAutor\nVPraze dne ....20..04. 2016 . .\nprojektová firma D-PLUS PROJEKTOVÁ A INŽENÝRSKÁ a.s., coby zaměstnavatel autora\n\n \n\nvykonávající majetková práva autora tímto potvrzuje, že si je vědoma smlouvy mezi Českou\nrepublikou — Ministerstvem zemědělství a zhotovitelem vybraným v rámci veřejné zakázky ze dne\nA 1 4YE...čj.16752-2016-MZE12134, jejímž předmětem bylo mimo jiné vypracování díla\nautorem, který je zaměstnancem projektové firmy, a projektová firma souhlasí s poskytnutím výhradní\nlicence či přímo poskytuje výhradní licenci k dílu České republice — Ministerstvu zemědělství\n\nd plus\nPROJEKTOVÁ A INŽENÝRSKÁ A.S. ©\nSOKOLOVSKÁ 16/45A\n186 00 PRAHA 8\n\n \n\n  \n\nMZe tímto prohlašuje, že\n\n- prohlášení Autora bere souhlasně na vědomí a\n\n- přijímá veškerá z něho plynoucí licenč

Wall time: 50.5 s


In [35]:
%%time

for filename in filenames:
    display(pytesseract.image_to_string(filename, lang='ces'))

'16752/2016-MZE-12134\n\np oso MAN\n000244093426\n\n  \n\n|\n\n|\n\n|\n\nSmlouva o dílo č. 256-2016-12134\nuzavřená podle $ 2586 a násl. zákona č. 89/2012 Sb., občanského zákoníku (dále jen\n\n„občanský zákoník“) ve spojení s $ 2631 občanského zákoníku\n\nmezi\n\nČeská republika — Ministerstvo zemědělství\nse sídlem: Těšnov 65/17, 110 00 Praha 1 — Nové Město\n\nIČ: 00020478\n\nDIČ: CZ00024478\n\nBankovní spojení: Česká národní banka Praha 1\n\nČíslo účtu: 1226-901118740 7226007/0270  OPRAVÁVO ŽE DVE\nzastoupenou Ing. Jiří Boháčkem, ředitelem odboru vnitřní správy\n\n(dále jen „objednatel“)\n\na společností\n\nD-PLUS PROJEKTOVÁ A INŽENÝRSKÁ a.s.\n\nse sídlem: Sokolovská 45/16, 186 00 Praha 8\n\nIČ: 26760312\n\nDIČ: CZ26760312\n\nSpolečnost je registrována u Městského soudu v Praze, oddíl B, vložka 8111\nBankovní spojení: ČSOB a.s., Praha 1\n\nčíslo účtu: 581094323/0300\n\nzastoupenou jednatelem společnosti\n\n(dále jen „zhotovitel “ nebo „projektant“)\n\nČlánek 1\n\nPředmět smlouvy\n\nP

'jen „DPS“) dle S 1 odst. 1 písm. f) vyhlášky 499/2006 Sb. (bude použita pro výběr\ndodavatele stavby) to vše na investiční akci „Projektová činnost pro rekonstrukci\nstřešního pláště budovy Jiráskova 1320 Rychnov nad Kněžnou“, autorský dozor výše\nuvedené investiční akce a zajištění průkazu energetické náročnosti budovy v souladu\ns příslušnými ustanoveními zákona č. 183/2006 Sb., o územním plánování a stavebním\nřádu (stavební zákon), ve znění pozdějších předpisů (dále jen „stavební zákon“), včetně\nprávních předpisů, které stavební zákon provádějí (veškeré výše uvedené společně dále\njen jako „dílo“).\n\nRozsah a etapy díla:\n\n1. Stavebně technické průzkumy, digitalizace a ověření stávajícího stavu, zpracování\nkoncepce řešení, DSP (V rozsahu pro výběr zhotovitele stavby — položkový rozpočet a\nvýkaz výměr) včetně pravomocného stavebního povolení, hodnocení ekonomického\npřínosu fotovoltaické folie\n\n1.1. Zajištění stavebně technického průzkumu\n\n- © zaměření předmětných prostor\

'zhotovitele stavby, kdy bude povinností projektanta posouzení rozpočtů v nabídkách\nuchazečů — zejména z pohledu problematiky mimořádně nízké nabídkové ceny, zda\njsou řádně vyplněny všechny položky atp.\n\n1.5. Autorský dozor projektanta v průběhu výstavby (dále jen „AD“)\n\nVýkon autorského dozoru projektanta bude spočívat zejména v poskytování\nvysvětlení potřebných pro vypracování dodavatelské dokumentace nebo pro plynulost\nstavby, dohledu nad dodržením dokumentace pro provádění stavby s přihlédnutím\nk podmínkám stavebního povolení a posuzování návrhů dodavatelů na změny a\nodchylky oproti této dokumentaci, zejména z pohledu dodržení technicko-\nekonomických parametrů stavby, architektonické koncepce, v povinné účasti na\nodevzdání a převzetí stavby, v povinné účasti na kontrolních dnech stavby apod.\n\nSoučástí ceny je tak nepřetržité provádění AD, a to do předání stavebního\ndíla/stavby, které je předmětem autorského dozoru. Předáním stavebního díla se\nrozumí dílo předané obj

'Vypracování projektantem 6 paré kompletní projektové dokumentace DSP a zajištění\nstavebního povolení (IČ) (ve smyslu nabytí právní moci) projektantem (dále jen jako\n„část díla 2“).\n\nVypracování projektantem 6 paré projektové dokumentace DPS (dále jen jako „část díla\n3).\n\nZajištění kolaudačního souhlasu projektantem (IČ) (ve smyslu nabytí právní moci tohoto\nrozhodnutí) a autorského dozoru projektanta (AD) v rozsahu a délce trvání uvedené\nv 1.5. (dále jen jako „část díla 4“).\n\nČlánek 2\nPovinnosti objednatele\n\n2.1. Objednatel se zavazuje, že poskytne zhotoviteli potřebnou součinnost při plnění této\nsmlouvy.\n\n2.2. Objednatel se zavazuje předat zhotoviteli všechny nezbytné podklady vztahující se\nk řešenému území.\n\n2.3. Případná vyjádření k projektům v průběhu prací se zavazuje objednatel předat\nzhotoviteli v přiměřených lhůtách, nikoliv však kratší Ihůtě než 5 pracovních dní\n\n2.4. Objednatel se zavazuje oznámit zhotoviteli včas překážky na straně objednatele,\nbráníc

'v 1.5. Smlouvy.\n\n3.5. V případě, že v rámci kontroly předjímané v čl. 3.4. bude konstatováno objednatelem\nporušení některé z povinností projektanta, předjímané smluvně či zákonem, pořídí o této\nskutečnosti objednatel zápis, jehož součástí může být i lhůta stanovená objednatelem\npro odstranění výše uvedené porušované povinnosti.\n\nČlánek 4\nDoba plnění\n\n4.1. Zhotovitel se zavazuje, že dílo, resp. jednotlivé části díla budou převzaty\nobjednatelem v souladu s touto Smlouvou v termínech níže uvedených a definovaných.\n\na) část díla 1 - do dvou týdnů od podpisu smlouvy druhou smluvní stranou\n\nb) část díla 2 - do 2 měsíců od podpisu smlouvy s podáním stavebního povolení\nna příslušné úřady\n\nc) část díla 3 - do jednoho měsíce od vydání pravomocného rozhodnutí o vydání\nstavebního povolení\n\nd) část díla 4 — do vydání pravomocného kolaudačního souhlasu, autorský dozor\npo dobu výstavby\n\n4.2. Termíny mohou být upraveny pouze po vzájemné dohodě, která musí být\nprovedena písemn

...

'v rozpracovaném stavu, a veškeré související dokumentace, objednatelem za podmínek\nstanovených touto smlouvou a autorským zákonem. Právo dílo užít podle předchozí věty\nzahrnuje všechna oprávnění dle ustanovení $ 12 zákona č. 121/2000 Sb., o právu\nautorském, o právech souvisejících s právem autorským a o změně některých zákonů\n(autorský zákon). Zhotovitel se zavazuje, že sám nepoužije ani neposkytne žádné třetí\nosobě bez předchozího písemného souhlasu objednatele práva kužití díla, resp.\njakékoliv části díla, provedeného dle této smlouvy. Právo dílo užít v rozsahu výše\nuvedeném vzniká objednateli předáním díla. Předáním dílčích částí díla vzniká\nobjednateli právo užít dílčí část díla v rozsahu předání této dílčí části.\n\n10.2. Smluvní strany sjednávají, že vlastnické právo k technickým dokumentacím, které\ntvoří součást díla, resp. část díla, jakož i všechny ostatní hmotné podklady, na kterých je\ndílo, resp. část díla vyjádřeno a které budou předány objednateli na základě tét

'V případě objednatele:\n\nČeská republika - Ministerstvo zemědělství\n\nTěšnov 65/17 , Praha 1 — Nové Město\n\nIng. Jiří Boháček, ředitel odboru vnitřní správy\n\ne-mail: jiri.bohacekAmze.cz\n\nV případě zhotovitele:\n\nD-PLUS PROJEKTOVÁ A INŽENÝRSKÁ a.s.\n\nSokolovská 45/16, Praha 8\n\nve věcech smluvních: Ing. Karel Janoch, předseda představenstva\n\ne-mail: karel.janoch©Ad-plus.cz\n\nve věcech technických: Ing. Karel Janoch, předseda představenstva\n\ne-mail: karel.janoch ©Wd-plus.cz\n\nnebo na takovou jinou adresu, která bude smluvní straně, která je odesílatelem\noznámení, předem oznámena v souladu s tímto článkem.\n\nPřílohy:\n\nč. 1— Prohlášení autora projektové dokumentace DSP, DPS\n\nza zhotovitele\n\nd plus\n\nAROJEKTOVÁ A mževýnaná A2 ©\n:OKOLOVSKÁ 16/45A.\n186 00 PRAHA 8\n\n \n\n16752/2016-MZE-12134\n\n-7 04, 2006\n\nV Praze, dne- K3\n\nza objednatele\nMINISTERSTVO\nZEMĚDĚLSTVÍ\n\n110.00 Prahz\n\n  \n\n   \n\nČeská republika — Ministerstvo zemědělství\nIng. Jiří Boháček\nř

"Prohlášení autora a výhradní bezúplatná licenční smlouva pro užití v neomezeném rozsahu\n\nJá, níže podepsaný: Ing. Karel Janoch,\n\nnar. 10. 7. 1964, bytem Černokostelecká 101/32, Praha 10, místem podnikání\nRČ 6407101910\n\n(dále jen „Autor“)\n\ntímto ve smyslu z.č. 89/2012 Sb., občanského zákoníku (dále jen „NOZ“), zejména ust. $ 2358 a násl.\na $ 2950 NOZ, a z.č. 121/2000 Sb., o právu autorském, o právech souvisejících s právem autorským\na o změně některých zákonů (dále jen „AZ“), zejména ust. $ 9 a násl. , $ 58 a/nebo $ 61 AZ,\n\nprohlašuji, že\n\n-jsem v rámci pracovněprávního vztahu k firmě D-PLUS PROJEKTOVÁ A INŽENÝRSKÁ a.s.\nvytvořil zaměstnanecké je Ji S 58 AZ a sohledem na smlouvu o dílo č.j 16752-2016-\nMZE12134 ze dne. uzavřenou mezi Českou republikou - Ministerstvem\nzemědělství, se sídlem m 17, Praha — Nové Město, PSČ: 110 00, IČ: 00020478 (dále jen\n„MZe“) a zhotovitelem vybraným v rámci VZMR „Projektová činnost pro M kgi ne\npláště budovy Jiráskova 1320, Rychnov nad 

'projekty a že spoléhá na pravost, úplnost, platnost a dlouhodobou účinnost tohoto prohlášení\n\nV Praze dne ... 20. 04,206\n\n \n\nAutor\nVPraze dne ....20..04. 2016 . .\nprojektová firma D-PLUS PROJEKTOVÁ A INŽENÝRSKÁ a.s., coby zaměstnavatel autora\n\n \n\nvykonávající majetková práva autora tímto potvrzuje, že si je vědoma smlouvy mezi Českou\nrepublikou — Ministerstvem zemědělství a zhotovitelem vybraným v rámci veřejné zakázky ze dne\nA 1 4YE...čj.16752-2016-MZE12134, jejímž předmětem bylo mimo jiné vypracování díla\nautorem, který je zaměstnancem projektové firmy, a projektová firma souhlasí s poskytnutím výhradní\nlicence či přímo poskytuje výhradní licenci k dílu České republice — Ministerstvu zemědělství\n\nd plus\nPROJEKTOVÁ A INŽENÝRSKÁ A.S. ©\nSOKOLOVSKÁ 16/45A\n186 00 PRAHA 8\n\n \n\n  \n\nMZe tímto prohlašuje, že\n\n- prohlášení Autora bere souhlasně na vědomí a\n\n- přijímá veškerá z něho plynoucí licenční oprávnění,\n\nčímž dochází k uzavření výhradní bezúplatné licenč

Wall time: 39.3 s


In [97]:
tikaExtractor.extract('../dokumenty/test_src/test_sub_01/test_contr_03/zeman_4.docx')

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nT 100 TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT\nMiloš Zeman (* 28. září 1944 Kolín) je český politik, ekonom, prognostik a\xa0třetí prezident České republiky. Úřadu se poprvé ujal složením slibu 8.\xa0března 2013 a\xa0druhé funkční období zahájil přesně o\xa0pět let později. \nV\xa0období 1993–2001 působil jako předseda České strany sociálně demokratické. V\xa0letech 1996–1998 zastával funkci předsedy Poslanecké sněmovny Parlamentu České republiky a\xa0následující čtyři roky byl předsedou sociálnědemokratické vlády menšinového charakteru, jejíž existence byla umožněna na základě tzv.\xa0„opoziční smlouvy“ s\xa0Občanskou demokratickou stranou. \nPo neúspěšné kandidatuře na prezidenta České republiky v\xa0roce 2003 odešel na sedm let z\xa0politiky. V\xa0březnu 2010 se stal předsedou nově založené Strany Práv Občanů ZEMANOVCI. Na funkci rezignoval po parlamentních volbác

In [99]:
tesseractExtractor.extract('../dokumenty/test_src/test_sub_01/test_contr_03/zeman_4.pdf')

'T 100 TTTTTTTTTTTTTUTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT\n\nMiloš Zeman (* 28. září 1944 Kolín) je český politik, ekonom, prognostik a třetí prezident\nCeské republiky. Uřadu se poprvé ujal složením slibu 8. března 2013 a druhé funkční období\nzahájil přesně o pět let později.\n\n \n\n \n\nV období 1993-2001 působil jako předseda České strany sociálně demokratické. V letech\n1996-1998 zastával funkci předsedy Poslanecké sněmovny Parlamentu České republiky\n\na následující čtyři roky byl předsedou sociálnědemokratické vlády menšinového charakteru,\njejíž existence byla umožněna na základě tzv. „opoziční smlouvy“ s Občanskou\ndemokratickou stranou.\n\n \n\n \n\nPo neúspěšné kandidatuře na prezidenta České republiky v roce 2003 odešel na sedm let\n\nz politiky. V březnu 2010 se stal předsedou nově založené Strany Práv Občanů\nZEMANOVCI. Na funkci rezignoval po parlamentních volbách 2010, ve kterých se strana\nnedostala do sněmovny..\n\nV přímé volbě v roce 2013 byl zvolen třetím p

In [82]:
machine.extract('../dokumenty/test_src/test_sub_01/test_contr_03/zeman_3.pdf')

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nT 100 TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT \n\nMiloš Zeman (* 28. září 1944 Kolín) je český politik, ekonom, prognostik a třetí prezident \n\nČeské republiky. Úřadu se poprvé ujal složením slibu 8. března 2013 a druhé funkční období \n\nzahájil přesně o pět let později.  \n\nV období 1993–2001 působil jako předseda České strany sociálně demokratické. V letech \n\n1996–1998 zastával funkci předsedy Poslanecké sněmovny Parlamentu České republiky \n\na následující čtyři roky byl předsedou sociálnědemokratické vlády menšinového charakteru, \n\njejíž existence byla umožněna na základě tzv. „opoziční smlouvy“ s Občanskou \n\ndemokratickou stranou.  \n\nPo neúspěšné kandidatuře na prezidenta České republiky v roce 2003 odešel na sedm let \n\nz politiky. V březnu 2010 se stal předsedou nově založené Strany Práv Občanů \n\nZEMANOVCI. Na funkci rezignoval po parlamentních volbách 2010, v

In [68]:
tessExtractor500dpi = PytesseractExtractionMachine(500)
tess500dpi = tessExtractor500dpi.extract('../dokumenty/test_src/test_sub_01/test_contr_03/zeman_3.pdf')
tess500dpi

'T100 TTTTITTTTTTT UT T TT TT TT TT TTTTT TT TT TT TTTTTTTTTTTTTTT\n\nMiloš Zeman (* 28. září 1944 Kolín) je český politik, ekonom, prognostik a třetí prezident\nCeské republiky. Uřadu se poprvé ujal složením slibu 8. března 2013 a druhé funkční období\nzahájil přesně o pět let později.\n\nV období 1993-2001 působil jako předseda České strany sociálně demokratické. V letech\n1996—1998 zastával funkci předsedy Poslanecké sněmovny Parlamentu České republiky\n\na následující čtyři roky byl předsedou sociálnědemokratické vlády menšinového charakteru,\njejíž existence byla umožněna na základě tzv. „opoziční smlouvy“ s Občanskou\ndemokratickou stranou.\n\nPo neúspěšné kandidatuře na prezidenta České republiky v roce 2003 odešel na sedm let\n\nz politiky. V březnu 2010 se stal předsedou nově založené Strany Práv Občanů\nZEMANOVCL. Na funkci rezignoval po parlamentních volbách 2010, ve kterých se strana\nnedostala do sněmovny“\n\n \n\nV přímé volbě v roce 2013 byl zvolen třetím prezidentem Čes

In [67]:
tessExtractor400dpi = PytesseractExtractionMachine(400)
tess400dpi = tessExtractor400dpi.extract('../dokumenty/test_src/test_sub_01/test_contr_03/zeman_3.pdf')
tess400dpi

'T100 TTTTTTTTTTTTULTTTTTVTTTT TU TT ET TTT EE TT TEL TT TVT TTTETTTTT\n\nMiloš Zeman (* 28. září 1944 Kolín) je český politik, ekonom, prognostik a třetí prezident\nCeské republiky. Uřadu se poprvé ujal složením slibu 8. března 2013 a druhé funkční období\nzahájil přesně o pět let později.\n\nV období 1993-2001 působil jako předseda České strany sociálně demokratické. V letech\n1996—1998 zastával funkci předsedy Poslanecké sněmovny Parlamentu České republiky\n\na následující čtyři roky byl předsedou sociálnědemokratické vlády menšinového charakteru,\njejíž existence byla umožněna na základě tzv. „opoziční smlouvy“ s Občanskou\ndemokratickou stranou.\n\nPo neúspěšné kandidatuře na prezidenta České republiky v roce 2003 odešel na sedm let\n\nz politiky. V březnu 2010 se stal předsedou nově založené Strany Práv Občanů\nZEMANOVCÍ. Na funkci rezignoval po parlamentních volbách 2010, ve kterých se strana\nnedostala do sněmovny. -n\n\nV přímé volbě v roce 2013 byl zvolen třetím prezidentem Č

In [66]:
tessExtractor300dpi = PytesseractExtractionMachine(300)
tess300dpi = tessExtractor300dpi.extract('../dokumenty/test_src/test_sub_01/test_contr_03/zeman_3.pdf')
tess300dpi

'T 100 TITTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTUTTTTUTTTTTTTTTT\n\nMiloš Zeman (* 28. září 1944 Kolín) je český politik, ekonom, prognostik a třetí prezident\nCeské republiky. Uřadu se poprvé ujal složením slibu 8. března 2013 a druhé funkční období\nzahájil přesně o pět let později.\n\n \n\n \n\nV období 1993-2001 působil jako předseda České strany sociálně demokratické. V letech\n1996-1998 zastával funkci předsedy Poslanecké sněmovny Parlamentu České republiky\n\na následující čtyři roky byl předsedou sociálnědemokratické vlády menšinového charakteru,\njejíž existence byla umožněna na základě tzv. „opoziční smlouvy“ s Občanskou\ndemokratickou stranou.\n\n \n\n \n\nPo neúspěšné kandidatuře na prezidenta České republiky v roce 2003 odešel na sedm let\n\nz politiky. V březnu 2010 se stal předsedou nově založené Strany Práv Občanů\nZEMANOVCI. Na funkci rezignoval po parlamentních volbách 2010, ve kterých se strana\nnedostala do sněmovny..\n\nV přímé volbě v roce 2013 byl zvolen třetím p

In [65]:
tessExtractor200dpi = PytesseractExtractionMachine(200)
tess200dpi = tessExtractor200dpi.extract('../dokumenty/test_src/test_sub_01/test_contr_03/zeman_3.pdf')
tess200dpi

'T 100 TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT\n\nMiloš Zeman (* 28. září 1944 Kolín) je český politik, ekonom, prognostik a třetí prezident\nCeské republiky. Úřadu se poprvé ujal složením slibu 8. března 2013 a druhé funkční období\nzahájil přesně o pět let později.\n\n \n\nV období 1993-2001 působil jako předseda České strany sociálně demokratické. V letech\n1996-1998 zastával funkci předsedy Poslanecké sněmovny Parlamentu České republiky\n\na následující čtyři roky byl předsedou sociálnědemokratické vlády menšinového charakteru,\njejíž existence byla umožněna na základě tzv. „opoziční smlouvy“ s Občanskou\ndemokratickou stranou.\n\nPo neúspěšné kandidatuře na prezidenta České republiky v roce 2003 odešel na sedm let\n\nz politiky. V březnu 2010 se stal předsedou nově založené Strany Práv Občanů\nZEMANOVCI Na funkci rezignoval po parlamentních volbách 2010, ve kterých se strana\nnedostala do sněmovny"\n\n \n\nV přímé volbě v roce 2013 byl zvolen třetím prezidentem Č

In [64]:
tessExtractor100dpi = PytesseractExtractionMachine(100)
tess100dpi = tessExtractor100dpi.extract('../dokumenty/test_src/test_sub_01/test_contr_03/zeman_3.pdf')
tess100dpi

'T 100 TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTITTTTTITTIT\n\nMiloš Zeman (* 28. září 1944 Kolín) je český politik, ekonom, prognostik a třetí prezident\nČeské republiky. Úřadu se poprvé ujal složením slibu 8. března 2013 a druhé funkční období\nzahájil přesně o pět let později.\n\nV období 1993-2001 působil jako předseda České strany sociálně demokratické, V letech\n1996-1998 zastával funkci předsedy Poslanecké sněmovny Parlamentu České republiky\n\na následující čtyři roky byl předsedou sociálnědemokratické vlády menšinového charakteru,\njejíž existence byla umožněna na základě tzv. „opoziční smlouvy“ s Občanskou\ndemokratickou stranou.\n\n \n\nPo neúspěšné kandidatuře na prezidenta České republiky v roce 2003 odešel na sedm let\n\nz politiky. V březnu 2010 se stal předsedou nově založené Strany Práv Občanů\nZEMANOVCI, Na funkci rezignoval po parlamentních volbách 2010, ve kterých se strana\nnedostala do sněmovny L\n\nV přímé volbě v roce 2013 byl zvolen třetím prezidentem Česk

In [100]:
%%time

tikaExtractor.extract('../dokumenty/test_src/test_sub_01/test_contr_03/zeman_5.docx')

Wall time: 189 ms


'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMiloš Zeman (* 28. září 1944 Kolín) je český politik, ekonom, prognostik a\xa0třetí prezident České republiky. Úřadu se poprvé ujal složením slibu 8.\xa0března 2013 a\xa0druhé funkční období zahájil přesně o\xa0pět let později. \nV\xa0období 1993–2001 působil jako předseda České strany sociálně demokratické. V\xa0letech 1996–1998 zastával funkci předsedy Poslanecké sněmovny Parlamentu České republiky a\xa0následující čtyři roky byl předsedou sociálnědemokratické vlády menšinového charakteru, jejíž existence byla umožněna na základě tzv.\xa0„opoziční smlouvy“ s\xa0Občanskou demokratickou stranou. \nPo neúspěšné kandidatuře na prezidenta České republiky v\xa0roce 2003 odešel na sedm let z\xa0politiky. V\xa0březnu 2010 se stal předsedou nově založené Strany Práv Občanů ZEMANOVCI. Na funkci rezignoval po parlamentních volbách 2010, ve kterých se strana nedostala do sněmovny.[1] \nV\xa0přímé v

In [152]:
%%time

tikaExtractor.extract('../dokumenty/test_src/test_sub_01/test_contr_03/zeman_5.pdf')

Wall time: 3.89 s


'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMiloš Zeman (* 28. září 1944 Kolín) je český politik, ekonom, prognostik a třetí prezident \n\nČeské republiky. Úřadu se poprvé ujal složením slibu 8. března 2013 a druhé funkční období \n\nzahájil přesně o pět let později.  \n\nV období 1993–2001 působil jako předseda České strany sociálně demokratické. V letech \n\n1996–1998 zastával funkci předsedy Poslanecké sněmovny Parlamentu České republiky \n\na následující čtyři roky byl předsedou sociálnědemokratické vlády menšinového charakteru, \n\njejíž existence byla umožněna na základě tzv. „opoziční smlouvy“ s Občanskou \n\ndemokratickou stranou.  \n\nPo neúspěšné kandidatuře na prezidenta České republiky v roce 2003 odešel na sedm let \n\nz politiky. V březnu 2010 se stal předsedou nově založené Strany Práv Občanů \n\nZEMANOVCI. Na funkci rezignoval po parlamentních volbách 2010, ve kterých se strana \n\nnedostala do sněmovny.[1]  \n\nV přímé volbě

In [157]:
%%time

tikaExtractor.extract('../dokumenty/test_src/test_sub_01/test_contr_02/smlouva.docx')

Wall time: 1.1 s


'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSmlouva XYZ\nPředmět\n\tNázev položky\n\tPopis položky\n\tCena bez DPH\n\tCena s\xa0DPH\n\n\tPoložka 1\n\tDetail 1\n\t10000 Kč\n\t12000 Kč\n\n\tPoložka 2\n\tDetail 2\n\t10\xa0000 Kč\n\t12\xa0000 Kč\n\n\tPoložka 3\n\tDetail 3\n\t1000000,1 Kč\n\t1200000,1 Kč\n\n\tPoložka 4\n\tDetail 4\n\t1\xa0000\xa0000,1 Kč\n\t1\xa0200\xa0000,1 Kč\n\n\n\n+ěščřžýáíéůúň\nPřednostně\n+ěščřžýáíéůúň\npřednostně\n+ěščřžýáíéůúň\npřednostně\n+ěščřžýáíéůúň\npřednostně\n+ěščřžýáíéůúň\npřednostně\n\n\n\n¨\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\nPoložka 5 Detail 1 10000 kč 12000 Kč\nPoložka 6 Detail 2 10000 Kč 12 000 kč\nPoložka 7 Detail 3 1000000,1 Kč 1200000,1 Kč\nPoložka 8 Detail 4 1000 000,1 Kč 1.200 000,1 Kč\n\n \n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nvěšězěváíé,\n\nPřednostně\n\n'

In [13]:
%%time

for i in range(10):
    print(i)
    tesseractExtractor.extract('../dokumenty/test_src/test_sub_01/test_contr_03/zeman_5.pdf')

0
1
2
3
4
5
6
7
8
9
Wall time: 24min 44s


In [14]:
%%time

for i in range(10):
    print(i)
    tikaExtractor.extract('../dokumenty/test_src/test_sub_01/test_contr_03/zeman_5.docx')

0
1
2
3
4
5
6
7
8
9
Wall time: 4.6 s


In [18]:
%%time

pages = pdf2image.convert_from_path('../dokumenty/test_src/test_sub_01/test_contr_03/zeman_5.pdf', 300)

Wall time: 18.3 s


In [19]:
%%time

extractions = []
for page in pages:
    extractions.append(pytesseract.image_to_string(page, lang='ces'))

Wall time: 2min 29s


In [141]:
machine = ExtractionMachine(save=True, filter_extracted=True)

In [145]:
%%time

dirs = ['../dokumenty/test_src/']
machine.extractFromDirs(dirs)

1/56: ../dokumenty/test_src/test_sub_01/test_contr_01/nadpis-ces.txt
2/56: ../dokumenty/test_src/test_sub_01/test_contr_01/nadpis.docx
3/56: ../dokumenty/test_src/test_sub_01/test_contr_01/nadpis.pdf
4/56: ../dokumenty/test_src/test_sub_01/test_contr_01/nadpis.PNG


2019-11-19 14:00:41,541 [MainThread  ] [WARNI]  Tika server returned status: 422


5/56: ../dokumenty/test_src/test_sub_01/test_contr_01/nadpis.txt
6/56: ../dokumenty/test_src/test_sub_01/test_contr_01/patice-ces.txt
7/56: ../dokumenty/test_src/test_sub_01/test_contr_01/patice.PNG
8/56: ../dokumenty/test_src/test_sub_01/test_contr_01/patice.txt
9/56: ../dokumenty/test_src/test_sub_01/test_contr_01/predmet-ces.txt
10/56: ../dokumenty/test_src/test_sub_01/test_contr_01/predmet.PNG
11/56: ../dokumenty/test_src/test_sub_01/test_contr_01/predmet.txt
12/56: ../dokumenty/test_src/test_sub_01/test_contr_01/renamed.pdf
13/56: ../dokumenty/test_src/test_sub_01/test_contr_01/smlouva o dílo, plošina.pdf
14/56: ../dokumenty/test_src/test_sub_01/test_contr_01/smlouva o dĂ­lo, ploĹˇina-01.jpg
15/56: ../dokumenty/test_src/test_sub_01/test_contr_01/smlouva o dĂ­lo, ploĹˇina-01.txt.txt
16/56: ../dokumenty/test_src/test_sub_01/test_contr_01/smlouva o dĂ­lo, ploĹˇina-02.jpg
17/56: ../dokumenty/test_src/test_sub_01/test_contr_01/smlouva o dĂ­lo, ploĹˇina-03.jpg
18/56: ../dokumenty/test_s

{}

In [144]:
machine2 = ExtractionMachine(extractor=CombinedDocExtractor2(), save=True, filter_extracted=True)

In [146]:
%%time

dirs = ['../dokumenty/test_src/']
machine2.extractFromDirs(dirs)

1/56: ../dokumenty/test_src/test_sub_01/test_contr_01/nadpis-ces.txt
2/56: ../dokumenty/test_src/test_sub_01/test_contr_01/nadpis.docx
3/56: ../dokumenty/test_src/test_sub_01/test_contr_01/nadpis.pdf
4/56: ../dokumenty/test_src/test_sub_01/test_contr_01/nadpis.PNG


2019-11-19 14:04:04,805 [MainThread  ] [WARNI]  Tika server returned status: 422


5/56: ../dokumenty/test_src/test_sub_01/test_contr_01/nadpis.txt
6/56: ../dokumenty/test_src/test_sub_01/test_contr_01/patice-ces.txt
7/56: ../dokumenty/test_src/test_sub_01/test_contr_01/patice.PNG
8/56: ../dokumenty/test_src/test_sub_01/test_contr_01/patice.txt
9/56: ../dokumenty/test_src/test_sub_01/test_contr_01/predmet-ces.txt
10/56: ../dokumenty/test_src/test_sub_01/test_contr_01/predmet.PNG
11/56: ../dokumenty/test_src/test_sub_01/test_contr_01/predmet.txt
12/56: ../dokumenty/test_src/test_sub_01/test_contr_01/renamed.pdf
13/56: ../dokumenty/test_src/test_sub_01/test_contr_01/smlouva o dílo, plošina.pdf
14/56: ../dokumenty/test_src/test_sub_01/test_contr_01/smlouva o dĂ­lo, ploĹˇina-01.jpg
15/56: ../dokumenty/test_src/test_sub_01/test_contr_01/smlouva o dĂ­lo, ploĹˇina-01.txt.txt
16/56: ../dokumenty/test_src/test_sub_01/test_contr_01/smlouva o dĂ­lo, ploĹˇina-02.jpg
17/56: ../dokumenty/test_src/test_sub_01/test_contr_01/smlouva o dĂ­lo, ploĹˇina-03.jpg
18/56: ../dokumenty/test_s

{}

In [148]:
machine3 = ExtractionMachine(extractor=TikaDocExtractor(), save=False, filter_extracted=True)

In [149]:
%%time

dirs = ['../dokumenty/test_src/']
machine3.extractFromDirs(dirs)

1/56: ../dokumenty/test_src/test_sub_01/test_contr_01/nadpis-ces.txt
2/56: ../dokumenty/test_src/test_sub_01/test_contr_01/nadpis.docx
3/56: ../dokumenty/test_src/test_sub_01/test_contr_01/nadpis.pdf
4/56: ../dokumenty/test_src/test_sub_01/test_contr_01/nadpis.PNG


2019-11-19 14:14:41,983 [MainThread  ] [WARNI]  Tika server returned status: 422


5/56: ../dokumenty/test_src/test_sub_01/test_contr_01/nadpis.txt
6/56: ../dokumenty/test_src/test_sub_01/test_contr_01/patice-ces.txt
7/56: ../dokumenty/test_src/test_sub_01/test_contr_01/patice.PNG
8/56: ../dokumenty/test_src/test_sub_01/test_contr_01/patice.txt
9/56: ../dokumenty/test_src/test_sub_01/test_contr_01/predmet-ces.txt
10/56: ../dokumenty/test_src/test_sub_01/test_contr_01/predmet.PNG
11/56: ../dokumenty/test_src/test_sub_01/test_contr_01/predmet.txt
12/56: ../dokumenty/test_src/test_sub_01/test_contr_01/renamed.pdf
13/56: ../dokumenty/test_src/test_sub_01/test_contr_01/smlouva o dílo, plošina.pdf
14/56: ../dokumenty/test_src/test_sub_01/test_contr_01/smlouva o dĂ­lo, ploĹˇina-01.jpg
15/56: ../dokumenty/test_src/test_sub_01/test_contr_01/smlouva o dĂ­lo, ploĹˇina-01.txt.txt
16/56: ../dokumenty/test_src/test_sub_01/test_contr_01/smlouva o dĂ­lo, ploĹˇina-02.jpg
17/56: ../dokumenty/test_src/test_sub_01/test_contr_01/smlouva o dĂ­lo, ploĹˇina-03.jpg
18/56: ../dokumenty/test_s

{'../dokumenty/test_src/test_sub_01/test_contr_01/nadpis-ces.txt': '\n\n\n\n\n\n\n\n\nSMLOUVA O DÍLO\nuzavřena podle $ 2586 a násl. zákona č. 89/2012 Sb., občanský zákoník (dále jen\n„občanský zákoník“)\nna akci: „„Zpracování projektové dokumentace a instalace vertikální\nzdvihací plošiny v Městské pečovatelské službě s denním stacionářem Louny\nvč. stavebních úprav a přípojky NN“\n�\n',
 '../dokumenty/test_src/test_sub_01/test_contr_01/nadpis.docx': '\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSMLOUVA O DÍLO\nuzavřena podle $ 2586 a násl. zákona č. 89/2012 Sb., občanský zákoník (dále jen\n„občanský zákoník“)\nna akci: „„Zpracování projektové dokumentace a instalace vertikální\nzdvihací plošiny v Městské pečovatelské službě s denním stacionářem Louny\nvč. stavebních úprav a přípojky NN“\n\n',
 '../dokumenty/test_src/test_sub_01/test_contr_01/nadpis.pdf': None,
 '../dokumenty/test_src/test_sub_01/test_contr_01/nadpis.PNG': '\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\